<center>
    <h1><b>Pretraining a Transformer Model from Scratch on an Amharic Dataset and Fine-Tuning for Amharic Hate Speech Recognition</b></h1>
</center>


## Table of Contents  

1. [Introduction](#introduction)  
2. [Importing Packages](#importing-packages)  
3. [Dataset Collection & Preprocessing](#dataset-collection--preprocessing)  
   - 3.1 [Data Collection](#data-collection)  
   - 3.2 [Data Cleaning](#data-cleaning)  
   - 3.3 [Tokenization](#tokenization)  
4. [Pretraining the Transformer Model](#pretraining-the-transformer-model)  
5. [Fine-Tuning for Hate Speech Recognition](#fine-tuning-for-hate-speech-recognition)  
6. [Evaluation](#evaluation)  
7. [Deployment on Mahder AI App](#deployment-on-mahder-ai-app)  
8. [Conclusion](#conclusion)  


## 1. Introduction  

In this notebook, I will pretrain a Transformer network on an Amharic dataset collected from a variety of Telegram channels, using the Masked Language Model (MLM). The primary objective of pretraining is to enable the model to learn contextualized word and phrase representations, thereby enhancing its understanding of language semantics. The Transformer’s self-attention mechanism plays a crucial role by allowing the model to dynamically weigh different parts of the input sequence, effectively capturing long-range dependencies in the data.  

After pretraining, I will fine-tune the model on a labeled dataset of hate speech and deploy the resulting model in the **Mahder AI** app.



## 2. Importing the Packages

Let's start by importing all the required libraries. 

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
import pandas as pd 
import json
import sentence


## 3. Dataset Collection & Preprocessing

### 3.1 Data Collection  

In order to pretrain the Transformer network from scratch, we will use **self-supervised learning**, which requires a large corpus of unlabeled text. We will apply a **Masked Language Model (MLM)** to pre-train the model.  

#### **Why Telegram Channels?**  
Telegram is the most widely used platform for information storage in Ethiopia. For this reason, I have chosen **Telegram channels** as the primary data source. Most of the selected channels are news channels, ensuring a diverse and rich dataset.  

#### **Data Collection Method**  
To collect the data, I used the **Telethon Python library** and the **Telegram API** to scrape text from selected channels.  

#### **Selected Telegram Channels**  
The dataset has been collected from the following Telegram channels:  

- [Tikvah Ethiopia](https://t.me/tikvahethiopia)  
- [Addis Standard Amharic](https://t.me/AddisstandardAmh)  
- [Tarikn Wedehuala](https://t.me/TariknWedehuala)  
- [Addis News](https://t.me/Addis_News)  
- [Zena 24 Now](https://t.me/zena24now)  
- [Tikvah University](https://t.me/TikvahUniversity)  
- [Tikvah Ethiopia Magazine](https://t.me/tikvahethmagazine)  
- [Tikvah Ethiopia Sport](https://t.me/tikvahethsport)  

#### **Accessing Collected Data**  
To access the code and all the raw data collected from each channel, visit the following GitHub repository:  
[GitHub Repository Link](https://github.com/your-repo-link-here)


### 3.2 Data Cleaning